# RAG連動チャットボット（LangGraph版）の構築

## 1.準備

In [ ]:
import uuid
from pathlib import Path
from typing import Any, Annotated, TypedDict

from dotenv import load_dotenv

from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_core.messages import AIMessageChunk, BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

load_dotenv()

MODEL_NAME = "gpt-5-mini"

## 2.テキストデータの用意

In [ ]:
TXT_DIR = Path("data/txt")
TXT_DIR.mkdir(parents=True, exist_ok=True)

# 出力がブレにくいように temperature は低め推奨
llm = ChatOpenAI(model=MODEL_NAME, temperature=0.2)
parser = StrOutputParser()

policy_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "あなたは企業の規程作成に詳しい専門家です。"
     "これから架空の会社の社内規則を日本語で作成します。"
     "実在の企業名・実在人物・個人情報は含めず、完全に架空の内容にしてください。"
     "出力は、章立て（見出し）と箇条書き、条項番号（例：第1条…）を含む読みやすいテキストにしてください。"
     "曖昧な表現を避け、例外条件・禁止事項・申請方法・期限などを明確にしてください。"
     "最後に『改定履歴：初版 2025-01-01』のような改定履歴を付けてください。"
     "余計な前置きは不要で、本文のみ出力してください。"
    ),
    ("human",
     "会社名：{company_name}\n"
     "業種：{industry}\n"
     "社員規模：{size}\n"
     "文書タイトル：{title}\n"
     "含めたいトピック：{topics}\n"
     "トーン：{tone}\n"
     "分量：{length}\n"
    ),
])

chain = policy_prompt | llm | parser

company_profile = {
    "company_name": "Aurora Works（オーロラワークス）",
    "industry": "BtoB SaaS（業務支援ソフトウェア）",
    "size": "従業員150名（国内中心、リモート併用）",
    "tone": "明確で実務的。例外条件や禁止事項も曖昧にしない。",
    "length": "A4で2〜3ページ相当（長すぎないが実務で使える粒度）",
}

docs_to_generate = [
    {
        "filename": "employee_handbook.txt",
        "title": "就業規則（抜粋）",
        "topics": "勤務時間、フレックスタイム、コアタイムの有無、リモート勤務、休暇、遅刻早退、兼業、評価、懲戒、相談窓口",
    },
    {
        "filename": "expense_policy.txt",
        "title": "経費精算規程",
        "topics": "交通費、出張、宿泊、日当、会食、領収書、精算期限、例外承認、違反時の扱い",
    },
    {
        "filename": "security_policy.txt",
        "title": "情報セキュリティ規程",
        "topics": "機密区分、パスワード、端末管理、持ち出し、クラウド利用、インシデント報告、禁止事項",
    },
]

for spec in docs_to_generate:
    text = chain.invoke({
        **company_profile,
        "title": spec["title"],
        "topics": spec["topics"],
    })
    (TXT_DIR / spec["filename"]).write_text(text, encoding="utf-8")
    print("Saved:", TXT_DIR / spec["filename"])


## 3.インデックスの作成

In [ ]:
# 1.読み込み（複数ファイルに対応）
loader = DirectoryLoader(
    path="data/txt",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True,
)
documents = loader.load()

# 2.分割（チャンク化）
# 日本語向けの separators を入れておくと安定しやすい
splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=150,
    separators=["\n\n", "\n", "。", "！", "？", "、", " ", ""],
)
chunks = splitter.split_documents(documents)

# 3.埋め込み、4.インデックス化
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(chunks)

retriever = vector_store.as_retriever(search_kwargs={"k": 4})

print(f"Loaded docs: {len(documents)} / chunks: {len(chunks)}")


## 4.RAG検索ツールの作成

In [ ]:
def format_docs(docs) -> str:
    lines = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("source", "unknown")
        src_name = Path(src).name if isinstance(src, str) else "unknown"
        lines.append(f"[{i}] source: {src_name}\n{d.page_content}")
    return "\n\n".join(lines) if lines else "（該当なし）"

@tool
def rag_search(query: str) -> str:
    """Aurora Works（架空）の社内規則TXTを検索し、関連箇所（抜粋）を返す。"""
    docs = retriever.invoke(query)
    return format_docs(docs)

tools = [rag_search]


## 5.LangGraphでRAGチャットボット化

In [ ]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# 共通プロンプト（Aurora Works専用であることを明示）
prompt_text = """
あなたは架空の会社「Aurora Works（オーロラワークス）」の社内規則に基づいて答えるアシスタントです。

ルール:
- Aurora Works の制度・就業・経費・セキュリティ等「会社に関する質問」は、必ず rag_search を使って根拠を確認してから答える
- 会社に関する質問ではない一般知識の質問は、ツールを使わずに普通に回答する
- rag_search を使った場合:
  - 回答末尾に参照番号（例：[1][2]）を付ける
  - 最後に「参照：」として、番号とファイル名を箇条書きで列挙する
- 根拠が見つからない場合は推測せず「資料内に根拠が見つかりません」と答える
- 取得した本文を大量に貼り付けず、要点をまとめて答える
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_text),
    MessagesPlaceholder(variable_name="messages"),
])

# LLMを2系統用意
# 1) 通常回答用（ツールなし）
model_plain = ChatOpenAI(model=MODEL_NAME, temperature=0.2)
chain_plain = prompt | model_plain

# 2) 会社質問 → ツール呼び出しを強制する用（ツールあり & required）
# ツールは rag_search 1つだけなので required が扱いやすい
model_force_tool = ChatOpenAI(model=MODEL_NAME, temperature=0.2).bind_tools(
    tools,
    tool_choice="required",
)
chain_force_tool = prompt | model_force_tool

def is_company_question(text: str) -> bool:
    """
    会社（Aurora Worksの社内規則）に関する質問かどうかを軽量判定する。
    ※学習用の簡易実装。運用ではログを見ながらキーワードを調整。
    """
    t = text.lower()

    keywords = [
        "オーロラワークス", "aurora works", "社内", "社内規則", "就業", "就業規則", "規程", "規則",
        "勤務", "労働", "出勤", "退勤", "休暇", "有給", "欠勤", "遅刻", "早退",
        "フレックス", "フレックスタイム", "コアタイム", "リモート", "在宅",
        "経費", "精算", "領収書", "出張", "宿泊", "日当", "会食",
        "セキュリティ", "機密", "パスワード", "端末", "持ち出し", "インシデント",
        "懲戒", "評価", "兼業", "副業", "相談窓口",
    ]
    return any(k in t for k in keywords)

def chatbot(state: State) -> dict[str, Any]:
    """
    修正ポイント:
    - 直近がユーザー発話なら「会社質問か？」を判定
      - 会社質問: chain_force_tool（ツール呼び出しを強制）
      - それ以外: chain_plain（普通に回答）
    - 直近がツール結果（ToolMessage）なら chain_plain で最終回答
      （ツール再呼び出しループ防止）
    """
    messages = state["messages"]
    last = messages[-1]

    last_type = type(last).__name__  # ToolMessage / HumanMessage / AIMessage など

    if last_type == "HumanMessage":
        user_text = getattr(last, "content", "") or ""
        if is_company_question(user_text):
            response = chain_force_tool.invoke(state)
        else:
            response = chain_plain.invoke(state)
    else:
        # ToolMessage を含む場合などは最終回答フェーズとして plain でまとめる
        response = chain_plain.invoke(state)

    return {"messages": [response]}

tool_node = ToolNode(tools)

# グラフ構築（構成は変えない）
builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_node("tools", tool_node)

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")

memory = InMemorySaver()
rag_agent = builder.compile(checkpointer=memory)

print("Ready.")


## 6.メインループ

In [ ]:
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    user_input = input("メッセージを入力:")
    if user_input.strip() == "":
        break

    print(f"質問：{user_input}")

    is_spinning = False

    for chunk, metadata in rag_agent.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config,
        stream_mode="messages",
    ):
        if isinstance(chunk, AIMessageChunk):
            if chunk.tool_call_chunks:
                if chunk.tool_call_chunks[-1]["name"] is not None:
                    print(f"[DEBUG]ツール呼び出し:{chunk.tool_call_chunks[-1]['name']}", flush=True)
                    is_spinning = True
                continue

            if chunk.content:
                if is_spinning:
                    print()
                    is_spinning = False
                print(chunk.content, end="", flush=True)

    print()

print("\n---ご利用ありがとうございました！---")